<a href="https://colab.research.google.com/github/Srini-UK/GenAI_Assignments/blob/main/LangGraph_PowerBI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
!pip install streamlit langgraph langchain google-generativeai msal requests fpdf matplotlib

# LangGraph Agent

In [ ]:
%%writefile agent_graph.py
import requests, json
import google.generativeai as genai
from langgraph.graph import StateGraph, END

genai.configure(api_key="YOUR_GEMINI_API_KEY")  # 🔧 Replace with your Gemini key

def fetch_metadata(state):
    headers = {"Authorization": f"Bearer {state['access_token']}"}
    url = f"https://api.powerbi.com/v1.0/myorg/groups/{state['workspace_id']}/reports/{state['report_id']}"
    report_info = requests.get(url, headers=headers).json()
    dataset_id = report_info.get("datasetId")
    sample_data = {}

    if dataset_id:
        tables_url = f"https://api.powerbi.com/v1.0/myorg/groups/{state['workspace_id']}/datasets/{dataset_id}/tables"
        tables_response = requests.get(tables_url, headers=headers).json()
        for table in [t["name"] for t in tables_response.get("value", [])][:2]:
            rows_url = f"https://api.powerbi.com/v1.0/myorg/groups/{state['workspace_id']}/datasets/{dataset_id}/tables/{table}/rows"
            rows_response = requests.get(rows_url, headers=headers).json()
            sample_data[table] = rows_response.get("value", [])[:5]

    return {"metadata": json.dumps({"report": report_info, "sample": sample_data}), "sample_data": sample_data}

def ask_gemini(state):
    model = genai.GenerativeModel("gemini-2.5-flash")
    prompt = f"""
    You are a BI assistant. Use the metadata below to answer the user's question.

    Question: {state['question']}
    Metadata: {state['metadata']}
    """
    response = model.generate_content(prompt).text.strip()
    return {"insight": response}

def build_agent():
    graph = StateGraph()
    graph.add_node("fetch_metadata", fetch_metadata)
    graph.add_node("ask_gemini", ask_gemini)
    graph.set_entry_point("fetch_metadata")
    graph.add_edge("fetch_metadata", "ask_gemini")
    graph.add_edge("ask_gemini", END)
    return graph.compile()

# Streamlit App

In [ ]:
%%writefile app.py
import streamlit as st
import matplotlib.pyplot as plt
import base64
from fpdf import FPDF
from msal import ConfidentialClientApplication
from agent_graph import build_agent

# 🔐 Replace with your actual values
TENANT_ID = "your-tenant-id"
CLIENT_ID = "your-client-id"
CLIENT_SECRET = "your-client-secret"
GEMINI_API_KEY = "your-gemini-api-key"

SCOPE = ["https://analysis.windows.net/powerbi/api/.default"]
app_auth = ConfidentialClientApplication(
    CLIENT_ID,
    authority=f"https://login.microsoftonline.com/{TENANT_ID}",
    client_credential=CLIENT_SECRET
)
ACCESS_TOKEN = app_auth.acquire_token_for_client(scopes=SCOPE).get("access_token")

# 🔧 Report options
report_options = {
    "IT Spend Analysis": {
        "workspace_id": "3750a12a-691c-4aac-8799-3ce40d992b53",
        "report_id": "b94a016d-a8dd-4db6-8a1d-85cc37d3c99c"
    }
}

# 🎛️ UI
st.set_page_config(page_title="Power BI Interactive Agent", page_icon="📊")
st.title("💬 Power BI Interactive Report Explorer")

selected_report = st.selectbox("Choose Report", list(report_options.keys()))
question = st.text_area("Ask a question about this report:")
mode = st.radio("Insight Style", ["Executive Summary", "Technical Breakdown"])

if "qa_history" not in st.session_state:
    st.session_state.qa_history = []

if st.button("Submit") and question.strip():
    with st.spinner("Thinking..."):
        agent = build_agent()
        state = {
            "question": question,
            "report_id": report_options[selected_report]["report_id"],
            "workspace_id": report_options[selected_report]["workspace_id"],
            "access_token": ACCESS_TOKEN
        }
        result = agent.invoke(state)
        st.session_state.qa_history.append({
            "question": question,
            "mode": mode,
            "insight": result["insight"],
            "sample_data": result.get("sample_data", {})
        })

# 🧠 Display History
if st.session_state.qa_history:
    st.subheader("🧠 Q&A History")
    for i, qa in enumerate(reversed(st.session_state.qa_history), 1):
        with st.expander(f"{qa['mode']} | Q{i}: {qa['question']}"):
            st.markdown("**💡 Insight:**")
            st.write(qa["insight"])

            if qa["sample_data"]:
                st.markdown("**📊 Charts:**")
                for table, rows in qa["sample_data"].items():
                    st.markdown(f"**Table: {table}**")
                    if rows and isinstance(rows[0], dict):
                        keys = list(rows[0].keys())
                        if len(keys) >= 2:
                            x = [str(row[keys[0]]) for row in rows]
                            y = [float(row[keys[1]]) if isinstance(row[keys[1]], (int, float)) else 0 for row in rows]
                            fig, ax = plt.subplots()
                            ax.bar(x, y)
                            ax.set_xlabel(keys[0])
                            ax.set_ylabel(keys[1])
                            ax.set_title(f"{table}: {keys[1]} by {keys[0]}")
                            st.pyplot(fig)

# 📤 Export
def generate_markdown_export(history):
    md = "# Power BI Q&A History\n\n"
    for i, qa in enumerate(history, 1):
        md += f"## Question {i}: {qa['question']}\n\n"
        md += f"**Mode:** {qa['mode']}\n\n"
        md += f"**Insight:**\n\n{qa['insight']}\n\n"
    return md

def download_button(label, content, filename):
    b64 = base64.b64encode(content.encode()).decode()
    href = f'<a href="data:file/txt;base64,{b64}" download="{filename}">{label}</a>'
    st.markdown(href, unsafe_allow_html=True)

if st.session_state.qa_history:
    st.subheader("📤 Export Options")
    markdown = generate_markdown_export(st.session_state.qa_history)
    download_button("⬇️ Download Markdown", markdown, "powerbi_insights.md")

    if st.button("⬇️ Generate PDF"):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt="Power BI Q&A History", ln=True, align="C")
        for i, qa in enumerate(st.session_state.qa_history, 1):
            pdf.set_font("Arial", style="B", size=12)
            pdf.multi_cell(0, 10, f"Question {i}: {qa['question']}")
            pdf.set_font("Arial", style="", size=11)
            pdf.multi_cell(0, 10, f"Mode: {qa['mode']}")
            pdf.multi_cell(0, 10, f"Insight:\n{qa['insight']}\n")
        pdf.output("powerbi_insights.pdf")
        st.success("PDF saved as powerbi_insights.pdf")

# Launch Streamlit

In [ ]:
!pip install pyngrok
from pyngrok import ngrok
import time

# Start Streamlit
get_ipython().system_raw('streamlit run app.py &')
time.sleep(5)

# Open tunnel
ngrok.set_auth_token("34tDSHi4VdZGx5IN7f8Gcp5uYs9_2PoXTHSNv67asekCtnQv2")  # 🔧 Replace with token
public_url = ngrok.connect(8501)
print("🔗 Streamlit app is live at:", public_url)